Importing Libraries

In [29]:
import gegravity as ge
import pandas as pd
import gme as gme

# Increase number of columns printed for a pandas DataFrame
pd.set_option("display.max_columns", None)
pd.set_option('display.width', 1000)

Loading dataset

In [30]:
gravity_data_location = "CPTPP_modified_main.csv" 
grav_data = pd.read_csv(gravity_data_location)

print(grav_data.head())

      exporter importer  year        trade             Y             E  pta  contiguity  common_language  lndist  international
0    Australia   Mexico  2021   2485957.01  3.420361e+08  5.065655e+08    1           0                0   14359              1
1       Canada   Mexico  2021  20453800.09  5.015389e+08  5.065655e+08    1           0                0    3604              1
2        Chile   Mexico  2021   1139794.39  9.467681e+07  5.065655e+08    1           0                0    7353              1
3        Japan   Mexico  2021   5062543.94  7.570663e+08  5.065655e+08    1           0                0   10791              1
4  New Zealand   Mexico  2021     85753.55  4.432529e+07  5.065655e+08    1           0                0   11103              1


Prepare Dataset and Econometric Inputs for GE Model

In [31]:
# Define GME Estimation Data
gme_data = gme.EstimationData(grav_data, # Dataset
                              imp_var_name="importer", # Importer column name
                              exp_var_name="exporter", # Exporter column name
                              year_var_name = "year",  # Year column name
                              trade_var_name="trade")  # Trade column name

# Create Gravity Model
gme_model = gme.EstimationModel(gme_data, # Specify data to use
                                lhs_var="trade",                               # dependent, "left hand side" variable
                                rhs_var=["pta","contiguity","common_language", # independent variables
                                         "lndist","international"],
                                fixed_effects=[["exporter"],["importer"]])     # Fixed effects to use

Estimate the Gravity Model with PPML

In [32]:
gme_model.estimate()
# Print econometric results table

print(gme_model.results_dict['all'].summary())

Estimation began at 12:02 PM  on Nov 16, 2023
Omitted Regressors: ['importer_fe_Vietnam']
Estimation completed at 12:02 PM  on Nov 16, 2023
                 Generalized Linear Model Regression Results                  
Dep. Variable:                  trade   No. Observations:                  400
Model:                            GLM   Df Residuals:                      356
Model Family:                 Poisson   Df Model:                           43
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.7401e+09
Date:                Thu, 16 Nov 2023   Deviance:                   3.4801e+09
Time:                        12:02:19   Pearson chi2:                 3.76e+09
No. Iterations:                    10   Pseudo R-squ. (CS):              1.000
Covariance Type:                  HC1                                         
                              coef    std err          z      P>|z|   

/home/debanjan/.local/lib/python3.10/site-packages/gme/estimate/_ppml_estimation_and_diagnostics.py:306: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  diagnostics = overfit_column.append(exclusion_column)


Define a GE Model

In [33]:
# Define GE model
ge_model = ge.OneSectorGE(gme_model,                   # gme gravity model
                       year = "2021",               # Year to use for model
                       expend_var_name = "E",       # Expenditure column name
                       output_var_name = "Y",       # Output column name
                       reference_importer = "India",  # Reference importer
                       sigma = 5)                   # Elasticity of substitution

Diagonose Model Solvability

In [34]:
# The following commands are not required to define or solve the GE model but can help diagnose issues that arise if
# the model fails to solve.

##
# Check inputs
##""

# Test that the model system of equations is computable from the supplied data and parameters
test_diagnostics = ge_model.test_baseline_mr_function()
# See what is returned:
print(test_diagnostics.keys())
# Check the values of the model parameters computed from the baseline data, which should be numeric with no missing
# values
input_params = test_diagnostics['mr_params']
# Check one set of parameters, for example:
print(input_params['cost_exp_shr'])


##
# Check scaling of outward multilateral resistances (OMRs)
##

# Check for OMR rescale factors that results in convergence
rescale_eval = ge_model.check_omr_rescale(omr_rescale_range=3)
print(rescale_eval)
# From the tests, it looks like 10, 100, and 1000 are good candidate rescale factors based on the fact that
# the model solves (i.e. converges) and all three produce consistent solutions for the reference importer's
# outward multilateral resistance (OMR) terms (2.918).

dict_keys(['initial values', 'mr_params', 'function_value'])
[[1.00506318e-02 7.69547389e-04 2.44411641e-05 1.58447015e-03
  5.70992826e-04 1.56659271e-02 4.35465196e-03 8.59696619e-03
  3.43520902e-03 4.45645685e-03 6.86033311e-04 3.19973389e-03
  1.40735497e-04 3.17462356e-04 1.73830486e-03 1.16566879e-03
  2.26930380e-03 9.49756414e-03 1.87046117e-03 1.70419880e-03]
 [1.66511421e-03 9.01724018e-03 5.45820428e-05 2.91005974e-03
  5.07915250e-04 1.75034807e-02 4.28453140e-03 8.53903664e-03
  3.36487548e-03 5.09662924e-03 1.51656621e-03 1.51695667e-03
  3.09567797e-04 9.14544029e-04 1.86824038e-03 2.65139536e-03
  4.07571342e-03 1.71611849e-02 2.15668168e-03 3.63086367e-03]
 [1.45685818e-03 1.46945583e-03 3.29457289e-04 3.07757198e-03
  5.97405351e-04 1.48605535e-02 4.05028815e-03 8.05278372e-03
  3.22718740e-03 4.31988573e-03 2.13784044e-03 3.22036551e-03
  2.87674547e-04 3.41055631e-04 2.40859763e-02 1.22544559e-03
  4.12506507e-03 1.74378463e-02 1.78466838e-03 3.19388838e-03]
 [8.43

/home/debanjan/.local/lib/python3.10/site-packages/gegravity/OneSectorGE.py:558: UserWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warn(solved_mrs.message)



Trying OMR rescale factor of 0.01
Solving for baseline MRs...


/home/debanjan/.local/lib/python3.10/site-packages/gegravity/OneSectorGE.py:558: UserWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warn(solved_mrs.message)



Trying OMR rescale factor of 0.1
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 1
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 10
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 100
Solving for baseline MRs...
The solution converged.

Trying OMR rescale factor of 1000
Solving for baseline MRs...
The solution converged.
   omr_rescale omr_rescale (alt format)  solved                                            message  max_func_value  mean_func_value  reference_importer_omr
0        0.001                    10^-3   False  The iteration is not making good progress, as ...    3.870182e-02     1.686257e-04                1.257069
1        0.010                    10^-2   False  The iteration is not making good progress, as ...    3.781309e-02     1.958435e-04                1.310264
2        0.100                    10^-1    True                            The solution converged.    8.

Solving the BaseLine GE Model

In [35]:
ge_model.build_baseline(omr_rescale=100)
# Examine the solutions for the baselin multilateral resistances
print(ge_model.baseline_mr.head(20))

Solving for baseline MRs...
The solution converged.
             baseline omr  baseline imr
country                                
Australia        1.942819      1.070281
Brazil           1.840491      1.054386
Brunei           1.808422      1.091267
Canada           1.797318      1.051717
Chile            1.866900      1.064646
China            1.620557      0.894584
France           1.849611      0.968671
Germany          1.693060      0.964329
India            1.787515      1.000000
Italy            1.769935      1.024815
Japan            1.776661      1.013177
Malaysia         1.944037      1.025192
Mexico           1.774734      0.999787
New Zealand      1.991070      1.120847
Peru             1.878647      1.056136
Russia           1.869219      0.961785
Singapore        1.892056      1.043771
UK               1.840134      1.075487
USA              1.547719      1.004403
Vietnam          1.765015      0.977437


Solving the CounterFactual GE Model

In [36]:
# Create a copy of the baseline data
exp_data = ge_model.baseline_data.copy()

# Modify the copied data to reflect a counterfactual experiment in which India and CPTPP12 countries sign a preferential trade agreement (pta)

exp_data.loc[(exp_data["importer"] == "Australia") & (exp_data["exporter"] == "India"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "India") & (exp_data["exporter"] == "Australia"), "pta"] = 1

exp_data.loc[(exp_data["importer"] == "Brunei") & (exp_data["exporter"] == "India"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "India") & (exp_data["exporter"] == "Brunei"), "pta"] = 1

exp_data.loc[(exp_data["importer"] == "Canada") & (exp_data["exporter"] == "India"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "India") & (exp_data["exporter"] == "Canada"), "pta"] = 1

exp_data.loc[(exp_data["importer"] == "Chile") & (exp_data["exporter"] == "India"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "India") & (exp_data["exporter"] == "Chile"), "pta"] = 1

exp_data.loc[(exp_data["importer"] == "Japan") & (exp_data["exporter"] == "India"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "India") & (exp_data["exporter"] == "Japan"), "pta"] = 1

exp_data.loc[(exp_data["importer"] == "Malaysia") & (exp_data["exporter"] == "India"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "India") & (exp_data["exporter"] == "Malaysia"), "pta"] = 1

exp_data.loc[(exp_data["importer"] == "Mexico") & (exp_data["exporter"] == "India"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "India") & (exp_data["exporter"] == "Mexico"), "pta"] = 1

exp_data.loc[(exp_data["importer"] == "New Zealand") & (exp_data["exporter"] == "India"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "India") & (exp_data["exporter"] == "New Zealand"), "pta"] = 1

exp_data.loc[(exp_data["importer"] == "Peru") & (exp_data["exporter"] == "India"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "India") & (exp_data["exporter"] == "Peru"), "pta"] = 1

exp_data.loc[(exp_data["importer"] == "Singapore") & (exp_data["exporter"] == "India"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "India") & (exp_data["exporter"] == "Singapore"), "pta"] = 1

exp_data.loc[(exp_data["importer"] == "UK") & (exp_data["exporter"] == "India"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "India") & (exp_data["exporter"] == "UK"), "pta"] = 1

exp_data.loc[(exp_data["importer"] == "Vietnam") & (exp_data["exporter"] == "India"), "pta"] = 1
exp_data.loc[(exp_data["importer"] == "India") & (exp_data["exporter"] == "Vietnam"), "pta"] = 1

# Define the experiment within the GE model
ge_model.define_experiment(exp_data)
# Examine the baseline and counterfactual trade costs
print(ge_model.bilateral_costs.head())

##
# Simulate the counterfactual model
##
ge_model.simulate()
# Examine the counterfactual trade flows predicted by the model.
print(ge_model.bilateral_trade_results.head())

                     baseline trade cost  experiment trade cost  trade cost change (%)
exporter  importer                                                                    
Australia Australia             0.511798               0.511798                    0.0
          Brazil                0.043678               0.043678                    0.0
          Brunei                0.037968               0.037968                    0.0
          Canada                0.042951               0.042951                    0.0
          Chile                 0.082502               0.082502                    0.0
Solving for conditional MRs...
The solution converged.
Solving full GE model...
The solution converged.
                     baseline modeled trade  experiment trade  trade change (percent)
exporter  importer                                                                   
Australia Australia            6.592889e+07      6.592200e+07               -0.010436
          Brazil             

### Access and Export Results

In [37]:
##
# Retrieve many of the different sets of model results
##
# A collection of many of the key country-level results (prices, total imports/exports, GDP, welfare, etc.)
country_results = ge_model.country_results
# The bilateral trade results
bilateral_results = ge_model.bilateral_trade_results
# A wider selection of aggregate, country-level trade results
agg_trade = ge_model.aggregate_trade_results
# country multilateral resistance (MR) terms
mr_terms = ge_model.country_mr_terms
# Get the solver diaganoistics, which is a dictionary containing many types of solver diagnostic info
solver_diagnostics = ge_model.solver_diagnostics

##
# Export results
##
# Export the results to a collection of spreadsheet (.csv) files and add trade values in levels to the outputs.
ge_model.export_results(directory="./Sample_result",name="India_CPTPP_PTA_experiment", include_levels = True)
# It is also possible to add alternative country identifies such as full country names using the country_names argument.
# See the technical documentation for details